# Implementation of Homebrew Network

In [ ]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))
from itertools import product

# Data imports
import cv2
import plotly.express as px
import numpy as np
import mlflow
import matplotlib.pyplot as plt

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer, ConvolutionLayer, PoolingLayer, FlattenLayer
from loss import CategoricalCrossEntropyLoss

## TESTING 
import importlib
importlib.reload(model)
##

## Data loader

In [11]:
def load_set(pth):
    X = []
    y = []

    for _class in os.listdir(pth):
        for img_name in os.listdir(os.path.join(pth, _class)):
            img = cv2.imread(os.path.join(pth, _class, img_name), cv2.IMREAD_GRAYSCALE)
            X.append(img)
            y.append(_class)

    X = np.array(X)

    # Normalizing 
    X = (X - np.mean(X))/ np.mean(X)

    # Vectorizing X
    X = X.reshape(X.shape[0], -1)

    # encoding y
    class_lbl_to_int = dict([(y, i) for i, y in enumerate(sorted(set(y)))])
    y = np.array([class_lbl_to_int[_y] for _y in y]).astype(np.uint8)
    y = one_hot_encode_index(y, len(class_lbl_to_int))

    # shuffling 
    keys = np.array(range(X.shape[0]))
    np.random.shuffle(keys)
    X = X[keys]
    y = y[keys]


    return X, y


def load_plant_data(resolution=32):
    base_dir = os.path.join(os.pardir, 'data', f'Plant_leave_diseases_{resolution}')
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')

    # loading train data 
    X_train, y_train = load_set(train_dir)

    # loading validation data 
    X_validation, y_validation = load_set(validation_dir)

    return X_train, y_train, X_validation, y_validation


## Setting up experiments

In [13]:
# Parimiters to test 
resolutions = [32]
leanring_rate = [0.001, 0.002]
decay = [1e-4, 1e-3]
dropout = [0.2, 0.9, 0.5]
configs = [{
    'resolution': cfg[0],
    'learning_rate': cfg[1],
    'decay': cfg[2],
    'dropout': cfg[3],
    'max_epochs': 200
    
} for cfg in product(resolutions, leanring_rate, decay, dropout)]

In [ ]:
mlflow.set_experiment("Plant Leaf Disease")

X_train, y_train, X_validation, y_validation = load_plant_data()

for config in configs:
    
    # initialize model 
    mdl = model.Model(Adam(learning_rate=config['learning_rate'], decay=config['decay']),
                      CategoricalCrossEntropyLoss())

    # Config early stop 
    mdl.add_early_stop(25)

    # Defining architecture 
    mdl.set_sequence([
                ConvolutionLayer(3, 32, 3),
                ReLU(),
                PoolingLayer(32, 2),
                ReLU(),
                ConvolutionLayer(32, 16, 1),
                ConvolutionLayer(16, 16, 3),
                ReLU(),
                PoolingLayer(16, 2),
                ReLU(),
                ConvolutionLayer(16, 8, 1),
                ReLU(),
                FlattenLayer(),
                LinearLayer(8 * 28 * 28, 1568),
                ReLU(),
                Dropout(0.5),
                LinearLayer(1568, 128),
                ReLU(),
                LinearLayer(128, 10),
                Softmax()
                    ])


    with mlflow.start_run():
        mlflow.log_param('framework', 'homebrew')
        mlflow.log_param('data_split', '90/10')
        mlflow.log_param('type', 'FFNN')
        mlflow.log_params(config)
        mdl.train(X_train, y_train, epochs=config['max_epochs'], batch_size=64, validation=(X_validation, y_validation))
        